In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from fbprophet import Prophet
import numpy as np
import os
from time import time
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

#mute unimportant diagnostic messages
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger().setLevel(logging.ERROR)

In [3]:
df = pd.read_csv('dsa_daily_jobs_alldates.csv', sep=',', header=0, parse_dates=[0], dayfirst=True)
df.tail()

,ds,y
2611,2019-02-24,87
2612,2019-02-25,202
2613,2019-02-26,85
2614,2019-02-27,571
2615,2019-02-28,609


In [4]:
df.dtypes

ds    datetime64[ns]
y              int64
dtype: object

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2616 entries, 0 to 2615
Data columns (total 2 columns):
ds    2616 non-null datetime64[ns]
y     2616 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 41.0 KB


In [6]:
from datetime import datetime
from datetime import timedelta

In [7]:
# set empty output list
output = [] ##SHOULD THIS BE A NEW PANDAS DATAFRAME? 
# ALTERNATIVE: fcst_all = pd.DataFrame() - initialise pd dataframe and concatenate in the loop - pd.concat(fcst_all)

# set INITIAL start date to run the model from the beginning of dataset 
# 2012-01-01
train_start_date = df['ds'].min()

# set the TRAIN span days - this length of time is CONSTANT but shifts 1 day per iteration
# 1886 days is 2012-01-01 - 2017-02-28, which leaves 1 year for testing and 1 year for 1 day increments 365 times
train_span = timedelta(days=1885)

# 2017-02-28
train_end_date = train_start_date + train_span

# TRAIN DATE RANGE - this range is constant at 1886 days but the dates that it covers shifts by one day each iteration
train_range = df[df['ds'].between(train_start_date, train_end_date)]



# set start date for first TEST period - training start date + 1 day
# 2017-03-01
test_start_date = train_end_date + timedelta(days=1)

# set the TEST span days - this length of time is CONSTANT but shifts along the date axis by 1 day per iteration
test_span = timedelta(days=364) 

# 2018-02-28 --- 365 days of testing
test_end_date = test_start_date + test_span

# TEST DATE RANGE
# test_range = pd.date_range(start = test_start_date, periods = 365)
test_range = df[df['ds'].between(test_start_date, test_end_date)]



# set the number of Model run iterations (will be set to 365 once confident it's working)
M = 5

# initialise iterator at zero
i = 0

# run the loop
while i < M:
    train =  train_range + timedelta(days=i)  # increase training range by i days
    m = Prophet().fit(train) # instantiate model and fit the training data
    test = test_range + timedelta(days=i) #increase test_span range by i days
    future = m.make_future_dataframe(periods=test) # make a future dataframe to store forecast predictions
    forecast = m.predict(future) # make forecast predictions on test range
    output.append(forecast['yhat']) # save modeling outputs to list or dataframe created above
    output.append(np.sqrt(mean_squared_error(test['y'], test['yhat']))) # Save RMSE
    output.append(mean_absolute_error(test['y'], test['yhat'])) # Save MAE
    output.append(np.mean(np.abs((test['y'] - fc_test['yhat'] / test['y'])) * 100)) # Save MAPE
    print(output)
    
    i += 1

TypeError: unsupported operand type(s) for +: 'numpy.ndarray' and 'Timedelta'